# Setup & Creation

In [6]:
import polars as pl
import numpy as np

# Constants
num_items = 5000
num_suppliers = 20
num_countries = 5
num_factory_demand_rows = 45000
num_target_volume_rows = num_suppliers * num_countries

# Create dummy ITEM LIST dataset
item_list_data = {
    'PDM MTL NUM': [f'MTL{i:04d}' for i in range(1, num_items + 1)],
    'NAME': [f'Item{i:04d}' for i in range(1, num_items + 1)],
    'ITEM TYPE': [f'Type{i%10}' for i in range(1, num_items + 1)],
    'STATE': ['New'] * num_items,
    'PDM SPL MTL NAME': [f'Material{i:04d}' for i in range(1, num_items + 1)],
    'PCX SPL MTL ID': [f'PCX{i:04d}' for i in range(1, num_items + 1)],
    'PCX MTL ID': [f'PCXM{i:04d}' for i in range(1, num_items + 1)],
    'SUPPLIER': [f'Supplier{i%num_suppliers + 1}' for i in range(1, num_items + 1)],
    'SUPPLIER COUNTRY': [f'Country{i%num_countries + 1}' for i in range(1, num_items + 1)],
    'PDM SPL MTL ': [f'SPLMTL{i:04d}' for i in range(1, num_items + 1)],
    'PP TYPE': [f'Type{i%3 + 1}' for i in range(1, num_items + 1)],
    'CAPACITY CODE': [f'Cap{i%3 + 1}' for i in range(1, num_items + 1)]
}
item_list_df = pl.DataFrame({k: v if isinstance(v, list) else v.tolist() for k, v in item_list_data.items()})

# Create dummy PRICE LIST dataset
price_list_data = {
    'PDM MTL NUM': [f'MTL{i:04d}' for i in range(1, num_items + 1)],
    'ITEM': [f'Item{i:04d}' for i in range(1, num_items + 1)],
    'SUPPLIER': [f'Supplier{i%num_suppliers + 1}' for i in range(1, num_items + 1)],
    'PDM SPL MTL NAME': [f'Material{i:04d}' for i in range(1, num_items + 1)],
    'PCX SPL MTL ID': [f'PCX{i:04d}' for i in range(1, num_items + 1)],
    'PCX MTL ID': [f'PCXM{i:04d}' for i in range(1, num_items + 1)],
    'BID PRICE': np.random.uniform(10, 100, num_items).tolist(),
    'SUPPLIER ': [f'Supplier{i%num_suppliers + 1}' for i in range(1, num_items + 1)],
    'SUPPLIER COUNTRY': [f'Country{i%num_countries + 1}' for i in range(1, num_items + 1)],
    'PDM SPL MTL NUM': [f'SPLMTL{i:04d}' for i in range(1, num_items + 1)],
    'PP TYPE': [f'Type{i%3 + 1}' for i in range(1, num_items + 1)],
    'CAPACITY CODE': [f'Cap{i%3 + 1}' for i in range(1, num_items + 1)]
}
price_list_df = pl.DataFrame({k: v if isinstance(v, list) else v.tolist() for k, v in price_list_data.items()})

# Create dummy FACTORY DEMAND dataset
factory_demand_data = {
    'PDM MTL NUM': [f'MTL{i%num_items + 1:04d}' for i in range(1, num_factory_demand_rows + 1)],
    'PCX MTL ID': [f'PCXM{i%num_items + 1:04d}' for i in range(1, num_factory_demand_rows + 1)],
    'VOLUME': np.random.choice([0, 1000, 2000], num_factory_demand_rows).tolist(),
    'FACTORY CODE': [f'Factory{i%60 + 1}' for i in range(1, num_factory_demand_rows + 1)]
}
factory_demand_df = pl.DataFrame({k: v if isinstance(v, list) else v.tolist() for k, v in factory_demand_data.items()})

# Create dummy TARGET VOLUME dataset
target_volume_data = {
    'SUPPLIER': [f'Supplier{i%num_suppliers + 1}' for i in range(1, num_target_volume_rows + 1)],
    'SUPPLIER COUNTRY': [f'Country{i%num_countries + 1}' for i in range(1, num_target_volume_rows + 1)],
    'TARGET %': np.random.uniform(0.01, 0.05, num_target_volume_rows).tolist(),
    'TARGET % RANGE': np.random.uniform(0.01, 0.02, num_target_volume_rows).tolist(),
    'SUPPLIER TYPE': [f'Type{i%3 + 1}' for i in range(1, num_target_volume_rows + 1)],
    'TOTAL VOLUME': np.random.randint(10000, 50000, num_target_volume_rows).tolist()
}
target_volume_df = pl.DataFrame({k: v if isinstance(v, list) else v.tolist() for k, v in target_volume_data.items()})

# Calculate Min and Max Target Volumes

In [7]:
# Calculate MIN TARGET VOLUME and MAX TARGET VOLUME
target_volume_df = target_volume_df.with_columns([
    ((target_volume_df['TARGET %'] - target_volume_df['TARGET % RANGE']) * target_volume_df['TOTAL VOLUME']).alias('MIN TARGET VOLUME'),
    ((target_volume_df['TARGET %'] + target_volume_df['TARGET % RANGE']) * target_volume_df['TOTAL VOLUME']).alias('MAX TARGET VOLUME')
])

target_volume_df.head()

SUPPLIER,SUPPLIER COUNTRY,TARGET %,TARGET % RANGE,SUPPLIER TYPE,TOTAL VOLUME,MIN TARGET VOLUME,MAX TARGET VOLUME
str,str,f64,f64,str,i64,f64,f64
"""Supplier2""","""Country2""",0.022332,0.016341,"""Type2""",49567,296.984777,1916.887001
"""Supplier3""","""Country3""",0.012922,0.019977,"""Type3""",40147,-283.238129,1320.805197
"""Supplier4""","""Country4""",0.020354,0.012104,"""Type1""",35714,294.647642,1159.185176
"""Supplier5""","""Country5""",0.024981,0.018913,"""Type2""",23022,139.694223,1010.537479
"""Supplier6""","""Country1""",0.033599,0.018734,"""Type3""",25134,373.611639,1315.354156


# Merge Item and Price Lists

In [8]:
# Merge ITEM LIST with PRICE LIST on 'PDM MTL NUM'
merged_df = item_list_df.join(price_list_df, on='PCX SPL MTL ID', how='inner')

# Prioritize by lower price
merged_df = merged_df.sort('BID PRICE')

In [9]:
merged_df.head(5)

PDM MTL NUM,NAME,ITEM TYPE,STATE,PDM SPL MTL NAME,PCX SPL MTL ID,PCX MTL ID,SUPPLIER,SUPPLIER COUNTRY,PDM SPL MTL,PP TYPE,CAPACITY CODE,PDM MTL NUM_right,ITEM,SUPPLIER_right,PDM SPL MTL NAME_right,PCX MTL ID_right,BID PRICE,SUPPLIER,SUPPLIER COUNTRY_right,PDM SPL MTL NUM,PP TYPE_right,CAPACITY CODE_right
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str
"""MTL2592""","""Item2592""","""Type2""","""New""","""Material2592""","""PCX2592""","""PCXM2592""","""Supplier13""","""Country3""","""SPLMTL2592""","""Type1""","""Cap1""","""MTL2592""","""Item2592""","""Supplier13""","""Material2592""","""PCXM2592""",10.019758,"""Supplier13""","""Country3""","""SPLMTL2592""","""Type1""","""Cap1"""
"""MTL2924""","""Item2924""","""Type4""","""New""","""Material2924""","""PCX2924""","""PCXM2924""","""Supplier5""","""Country5""","""SPLMTL2924""","""Type3""","""Cap3""","""MTL2924""","""Item2924""","""Supplier5""","""Material2924""","""PCXM2924""",10.070995,"""Supplier5""","""Country5""","""SPLMTL2924""","""Type3""","""Cap3"""
"""MTL4162""","""Item4162""","""Type2""","""New""","""Material4162""","""PCX4162""","""PCXM4162""","""Supplier3""","""Country3""","""SPLMTL4162""","""Type2""","""Cap2""","""MTL4162""","""Item4162""","""Supplier3""","""Material4162""","""PCXM4162""",10.073393,"""Supplier3""","""Country3""","""SPLMTL4162""","""Type2""","""Cap2"""
"""MTL2229""","""Item2229""","""Type9""","""New""","""Material2229""","""PCX2229""","""PCXM2229""","""Supplier10""","""Country5""","""SPLMTL2229""","""Type1""","""Cap1""","""MTL2229""","""Item2229""","""Supplier10""","""Material2229""","""PCXM2229""",10.07564,"""Supplier10""","""Country5""","""SPLMTL2229""","""Type1""","""Cap1"""
"""MTL4021""","""Item4021""","""Type1""","""New""","""Material4021""","""PCX4021""","""PCXM4021""","""Supplier2""","""Country2""","""SPLMTL4021""","""Type2""","""Cap2""","""MTL4021""","""Item4021""","""Supplier2""","""Material4021""","""PCXM4021""",10.084515,"""Supplier2""","""Country2""","""SPLMTL4021""","""Type2""","""Cap2"""


# Apply LOCAL FOR LOCAL logic (matching supplier and factory country)

In [12]:
merged_df = merged_df.with_columns(
    (merged_df['SUPPLIER COUNTRY'] == merged_df['SUPPLIER COUNTRY_right']).alias('LOCAL_FOR_LOCAL')
)
merged_df = merged_df.sort(['BID PRICE', 'LOCAL_FOR_LOCAL'], descending=[False, True])

merged_df.head()

PDM MTL NUM,NAME,ITEM TYPE,STATE,PDM SPL MTL NAME,PCX SPL MTL ID,PCX MTL ID,SUPPLIER,SUPPLIER COUNTRY,PDM SPL MTL,PP TYPE,CAPACITY CODE,PDM MTL NUM_right,ITEM,SUPPLIER_right,PDM SPL MTL NAME_right,PCX MTL ID_right,BID PRICE,SUPPLIER,SUPPLIER COUNTRY_right,PDM SPL MTL NUM,PP TYPE_right,CAPACITY CODE_right,LOCAL_FOR_LOCAL
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,bool
"""MTL2592""","""Item2592""","""Type2""","""New""","""Material2592""","""PCX2592""","""PCXM2592""","""Supplier13""","""Country3""","""SPLMTL2592""","""Type1""","""Cap1""","""MTL2592""","""Item2592""","""Supplier13""","""Material2592""","""PCXM2592""",10.019758,"""Supplier13""","""Country3""","""SPLMTL2592""","""Type1""","""Cap1""",true
"""MTL2924""","""Item2924""","""Type4""","""New""","""Material2924""","""PCX2924""","""PCXM2924""","""Supplier5""","""Country5""","""SPLMTL2924""","""Type3""","""Cap3""","""MTL2924""","""Item2924""","""Supplier5""","""Material2924""","""PCXM2924""",10.070995,"""Supplier5""","""Country5""","""SPLMTL2924""","""Type3""","""Cap3""",true
"""MTL4162""","""Item4162""","""Type2""","""New""","""Material4162""","""PCX4162""","""PCXM4162""","""Supplier3""","""Country3""","""SPLMTL4162""","""Type2""","""Cap2""","""MTL4162""","""Item4162""","""Supplier3""","""Material4162""","""PCXM4162""",10.073393,"""Supplier3""","""Country3""","""SPLMTL4162""","""Type2""","""Cap2""",true
"""MTL2229""","""Item2229""","""Type9""","""New""","""Material2229""","""PCX2229""","""PCXM2229""","""Supplier10""","""Country5""","""SPLMTL2229""","""Type1""","""Cap1""","""MTL2229""","""Item2229""","""Supplier10""","""Material2229""","""PCXM2229""",10.07564,"""Supplier10""","""Country5""","""SPLMTL2229""","""Type1""","""Cap1""",true
"""MTL4021""","""Item4021""","""Type1""","""New""","""Material4021""","""PCX4021""","""PCXM4021""","""Supplier2""","""Country2""","""SPLMTL4021""","""Type2""","""Cap2""","""MTL4021""","""Item4021""","""Supplier2""","""Material4021""","""PCXM4021""",10.084515,"""Supplier2""","""Country2""","""SPLMTL4021""","""Type2""","""Cap2""",true


# Define Allocation Function

In [17]:
# Define a function for allocation based on target volume and factory demand
def allocate_volume(merged_df, target_volume_df, factory_demand_df):
    allocations = []
    
    for supplier in target_volume_df.iter_rows(named=True):
        supplier_name = supplier['SUPPLIER']
        min_volume = supplier['MIN TARGET VOLUME']
        max_volume = supplier['MAX TARGET VOLUME']
        supplier_df = merged_df.filter(pl.col('SUPPLIER_right') == supplier_name)
        
        allocated_volume = 0
        for row in supplier_df.iter_rows(named=True):
            if allocated_volume >= max_volume:
                break

            factory_options = factory_demand_df.filter(pl.col('PDM MTL NUM') == row['PDM MTL NUM'])
            for factory in factory_options.iter_rows(named=True):
                if allocated_volume >= max_volume:
                    break
                if factory['VOLUME'] > 0:
                    volume_to_allocate = min(max_volume - allocated_volume, factory['VOLUME'])
                    allocations.append({
                        'PCX SPL MTL ID': row['PCX SPL MTL ID'],
                        'PDM MTL NUM': row['PDM MTL NUM'],
                        'SUPPLIER': row['SUPPLIER_right'],
                        'FACTORY CODE': factory['FACTORY CODE'],
                        'ALLOCATION (UNITS)': volume_to_allocate,
                        'PRICE': row['BID PRICE']
                    })
                    allocated_volume += volume_to_allocate
                    factory['VOLUME'] -= volume_to_allocate
    
    return pl.DataFrame(allocations)

# Perform Allocation and Save Results

In [18]:
# Perform the allocation
allocation_df = allocate_volume(merged_df, target_volume_df, factory_demand_df)

# Output the allocation results in the same format as ALLOCATION RESULT file
allocation_result_columns = ['PCX SPL MTL ID', 'PDM MTL NUM', 'SUPPLIER', 'FACTORY CODE', 'ALLOCATION (UNITS)', 'PRICE']
allocation_df = allocation_df.select(allocation_result_columns)
allocation_df.head(10)

PCX SPL MTL ID,PDM MTL NUM,SUPPLIER,FACTORY CODE,ALLOCATION (UNITS),PRICE
str,str,str,str,f64,f64
"""PCX4021""","""MTL4021""","""Supplier2""","""Factory21""",1000.0,10.084515
"""PCX4021""","""MTL4021""","""Supplier2""","""Factory41""",916.887001,10.084515
"""PCX4162""","""MTL4162""","""Supplier3""","""Factory42""",1320.805197,10.073393
"""PCX0683""","""MTL0683""","""Supplier4""","""Factory23""",1159.185176,10.243182
"""PCX2924""","""MTL2924""","""Supplier5""","""Factory4""",1010.537479,10.070995
"""PCX2805""","""MTL2805""","""Supplier6""","""Factory25""",1000.0,10.332189
"""PCX2805""","""MTL2805""","""Supplier6""","""Factory45""",315.354156,10.332189
"""PCX0986""","""MTL0986""","""Supplier7""","""Factory26""",2000.0,10.142198
"""PCX0986""","""MTL0986""","""Supplier7""","""Factory46""",11.653521,10.142198


# Save the results to an Excel file

In [ ]:
allocation_df.write_excel('ALLOCATION_RESULT.xlsx')

print("Allocation results saved to ALLOCATION_RESULT.xlsx")